In [ ]:
import json
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.metrics import confusion_matrix


In [ ]:
# Load data from JSON file
with open('raceInfo.json') as f:
    data = json.load(f)

In [ ]:
# Create a DataFrame with the data
df = pd.DataFrame(data.items(), columns=['driver', 'results'])

# Expand the results column into separate columns
df = pd.concat([df.drop('results', axis=1), df['results'].apply(pd.Series)], axis=1)

# Transpose the DataFrame so that each row corresponds to a race result and each column corresponds to a driver
df = df.set_index('driver').T


In [ ]:
n_races = len(df)
train_ratio = 0.8

train_size = int(n_races*train_ratio)

X_train = np.array([np.array(df.iloc[:train_size, i]) for i in range(len(df.columns))])
y_train = np.array([np.array(df.iloc[train_size:, i]) for i in range(len(df.columns))])

X_test = np.array([np.array(df.iloc[train_size:, i]) for i in range(len(df.columns))])

model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(None, 1)))
model.add(Dense(n_races - train_size, activation='linear'))
model.compile(optimizer='adam', loss='mse')

X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)

model.fit(X_train, y_train, epochs=100)

In [ ]:
df

driver,Michael Schumacher,Juan Pablo Montoya,Kimi Räikkönen,Eddie Irvine,Mark Webber,Mika Salo,Alex Yoong,Pedro de la Rosa,David Coulthard,Jacques Villeneuve,...,Ralf Schumacher,Giancarlo Fisichella,Felipe Massa,Nick Heidfeld,Jenson Button,Olivier Panis,Allan McNish,Heinz-Harald Frentzen,Enrique Bernoldi,Anthony Davidson
0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,14.0,15.0,16.0,17.0,18.0,19.0,20.0,11.0,19.0,17.0
1,3.0,2.0,18.0,16.0,15.0,12.0,17.0,10.0,20.0,8.0,...,1.0,13.0,6.0,5.0,4.0,21.0,7.0,19.0,20.0,18.0
2,1.0,5.0,12.0,7.0,11.0,6.0,13.0,8.0,3.0,10.0,...,2.0,22.0,16.0,14.0,4.0,18.0,17.0,18.0,12.0,NaN
3,1.0,4.0,14.0,13.0,11.0,17.0,14.0,16.0,6.0,7.0,...,3.0,19.0,8.0,10.0,5.0,15.0,21.0,6.0,15.0,NaN
4,1.0,2.0,18.0,14.0,12.0,9.0,20.0,19.0,3.0,7.0,...,11.0,17.0,5.0,4.0,12.0,13.0,8.0,11.0,21.0,NaN
5,1.0,3.0,20.0,15.0,11.0,8.0,14.0,22.0,6.0,10.0,...,4.0,5.0,19.0,16.0,7.0,18.0,9.0,6.0,12.0,NaN
6,2.0,17.0,19.0,9.0,11.0,13.0,18.0,10.0,1.0,18.0,...,3.0,5.0,14.0,8.0,16.0,15.0,22.0,13.0,21.0,NaN
7,1.0,16.0,4.0,18.0,15.0,19.0,10.0,20.0,2.0,22.0,...,7.0,5.0,9.0,12.0,15.0,8.0,17.0,13.0,10.0,NaN
8,2.0,20.0,3.0,19.0,20.0,17.0,13.0,11.0,21.0,12.0,...,4.0,22.0,6.0,7.0,5.0,9.0,14.0,18.0,16.0,NaN
9,1.0,3.0,14.0,17.0,8.0,19.0,18.0,11.0,10.0,4.0,...,8.0,7.0,9.0,6.0,12.0,5.0,21.0,20.0,13.0,NaN


In [ ]:
predictions = model.predict(np.expand_dims(X_test, axis=2))
final_ranks = np.argsort(predictions[:, -1])

predicted_winner = df.columns[final_ranks[0]]
print("The predicted winner of the final race is:", predicted_winner)

1/1 [==============================] - 0s 198ms/step
The predicted winner of the final race is: Michael Schumacher


In [ ]:
predictions = model.predict(np.expand_dims(X_test, axis=2))
final_ranks = np.argsort(predictions[:, -1])

predicted_winner = df.columns[final_ranks[0]]
print("The predicted winner of the final race is:", predicted_winner)

actual_winner = df.iloc[train_size:, :].idxmin(axis=1).values[0]
accuracy = int(predicted_winner == actual_winner)
print("Accuracy of prediction:", accuracy)

1/1 [==============================] - 0s 31ms/step
The predicted winner of the final race is: Michael Schumacher
Accuracy of prediction: 1
